# Lab Assignment Six: Convolutional Network Architectures

## Catherine Magee, Morgan Mote, Luv Patel

In this lab, you will select a prediction task to perform on your dataset, evaluate a deep learning architecture and tune hyper-parameters. If any part of the assignment is not clear, ask the instructor to clarify. 

This report is worth 10% of the final grade. Please upload a report (one per team) with all code used, visualizations, and text in a rendered Jupyter notebook. Any visualizations that cannot be embedded in the notebook, please provide screenshots of the output. The results should be reproducible using your report. Please carefully describe every assumption and every step in your report.

### Dataset Selection

Select a dataset identically to lab two (images). That is, the dataset must be image data. In terms of generalization performance, it is helpful to have a large dataset of identically sized images. It is fine to perform binary classification or multi-class classification. You are not allowed to use MNIST, Fashion, MNIST, or the sklearn digits dataset.

##### Grading Rubric

- Preparation (3 points total)  

>[1.5 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

>[1.5 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice. 

- Modeling (6 points total)

>[1.5 points]  Setup the training to use data expansion in Keras (also called data augmentation). Explain why the chosen data expansion techniques are appropriate for your dataset. You should make use of Keras augmentation layers, like in the class examples.

>[2 points] Create a convolutional neural network to use on your data using Keras. Investigate at least two different convolutional network architectures and investigate changing one or more parameters of each architecture such as the number of filters. This means, at a  minimum, you will train a total of four models (2 different architectures, with 2 parameters changed in each architecture). Use the method of train/test splitting and evaluation metric that you argued for at the beginning of the lab. Visualize the performance of the training and validation sets per iteration (use the "history" parameter of Keras). Be sure that models converge. 

>[1.5 points] Visualize the final results of all the CNNs and interpret/compare the performances. Use proper statistics as appropriate, especially for comparing models. 

>[1 points] Compare the performance of your convolutional network to a standard multi-layer perceptron (MLP) using the receiver operating characteristic and area under the curve. Use proper statistical comparison techniques.  

- Exceptional Work (1 points total)

>You have free reign to provide additional analyses. One idea (required for 7000 level students): Use transfer learning with pre-trained weights for your initial layers of your CNN. Compare the performance when using transfer learning to your best model from above in terms of classification performance. 

##### CNN Rubric

- Metric Chosen
> Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

- Data Separation
>Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice.

- Create CNN and Use Expansion
>Create a convolutional neural network to use on your data using Keras. Setup the training to use data expansion in Keras. Explain why the chosen data expansion techniques are appropriate for your dataset.

- Build Two Different Architectures
>Investigate at least two different convolutional network architectures (and investigate changing some parameters of each architecture). Use the method of cross validation and evaluation metric that you argued for at the beginning of the lab. Visualize the performance of the training and validation sets per iteration (use the "history" parameter of Keras).

- Visualize and Compare
>Visualize the final results of the CNNs and interpret the performance. Use proper statistics as appropriate, especially for comparing models.

- Compare to MLP
>Compare the performance of your convolutional network to a standard multi-layer perceptron (MLP) using the receiver operating characteristic and area under the curve. Use proper statistical comparison techniques.

- Exceptional Work
>Required for 7000 level students: Use transfer learning to pre-train the weights of your initial layers of your CNN. Compare the performance when using transfer learning to training from scratch in terms of classification performance.

### Dataset Selection:

https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia

##### Content
The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal).

Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care.

For the analysis of chest x-ray images, all chest radiographs were initially screened for quality control by removing all low quality or unreadable scans. The diagnoses for the images were then graded by two expert physicians before being cleared for training the AI system. In order to account for any grading errors, the evaluation set was also checked by a third expert.

The normal chest X-ray depicts clear lungs without any areas of abnormal opacification in the image. Bacterial pneumonia  typically exhibits a "focal lobar consolidation", whereas viral pneumonia manifests with a more "diffuse interstitial pattern" in both lungs.

##### Acknowledgements
Data: https://data.mendeley.com/datasets/rscbjbr9sj/2

License: CC BY 4.0

Citation: http://www.cell.com/cell/fulltext/S0092-8674(18)30154-5

# DATASET PREPARATION

In [4]:
!pip install tensorflow

In [7]:
!pip install tensorflow_addons

     ------------------------------------- 729.9/729.9 kB 11.6 MB/s eta 0:00:00


In [8]:
import subprocess
import platform
import os
import itertools
import time
from IPython.display import display, HTML, Markdown, clear_output

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import (
    utils,
    models,
    layers,
    metrics,
    preprocessing,
    callbacks,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Reshape,
    Input,
    Dense,
    Dropout,
    Activation,
    Flatten,
    Conv2D,
    MaxPooling2D,
    average,
    Add,
    concatenate,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

import tensorflow_addons as tfa

from sklearn import metrics
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    make_scorer,
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    ConfusionMatrixDisplay,
    roc_curve,
    auc,
)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    cross_val_score,
)

from scipy.signal import savgol_filter

def clear_screen():
    time.sleep(2)
    print("Clearing screen...")
    time.sleep(2)
    clear_output()

clear_screen()

In [9]:
LIMIT_GPU_MEMORY = False

if LIMIT_GPU_MEMORY:
    # Limit GPU Memory Usage
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(
        physical_devices[0],
        False
    )
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = str(0)

### Evaluation Method

For evaluating our model's performance, we will be using recall. Between the two conditions, pneumonia is potentially very dangerous for the young, elderly, and people already diagnosed with a disease or respiratory condition. We want to consider false negatives (the model predicts pneumonia lesions when it is actually something else like bronchiectasis) as we evaluate whether our model is performing well or not.

Beyond recall, we use metrics such as accuracy, misclassificaiton, precision, recall, and F1-score to measure and compare algorithm performance.

- Accuracy is a measure of the overall correctness of a model's predictions, calculated as the ratio of the number of correct predictions to the total number of predictions.

- Misclass, or misclassification, refers to the instances where a model's prediction does not match the actual class label of a data point.

- Precision is a measure of how well a model correctly identifies positive instances among the predicted positive instances.

- Recall, also known as sensitivity or the true positive rate, is a measure of how well a model identifies all the positive instances among the actual positive instances.

- F1-score is the "harmonic mean" of precision and recall, providing a single value that balances the trade-off between precision and recall.

- Additionally, we graph the values of Loss and AUC (Area Under Curve) for comparisons.

We want to consider all of these metrics since not only do we want our models to perform well, but we want to achieve the highest true positive rate for both classes as possible. Given the importance of recognizing false positive and negatives, we need to measure and quantify the performance our our models by all of these metrics.

### Dataset Splitting

We have around 5900 observations, so the amount of data is not a concern. We also assessed the distribution of the class variable and found that the target class variable is evenly distributed.

We feel that the data is already well-structured and there is not a need to split the data into subsets or "folds". The training and testing folders appear to be representative of the overall population of images that the model is expected to encounter.

We have decided to proceed with the current testing and training datasets as they are, without performing cross-validation. We feel that not using cross-validation can be justified for a few reasons:

For one, the dataset size is large enough and representative of the overall population of images, so it may not be necessary to perform cross-validation as the model can learn from the full dataset without having to split it into subsets. Additionally, as the training and testing folders have a balanced distribution of images across all classes, then cross-validation may not be necessary as the model canlearn again from the full dataset without having to split it into subsets.

We additonally decided that it would be acceptable to use grayscale versions of the images for the CNN models, since x-ray images are not depicted as RGB to begin with.

In [11]:
# Load the image dataset

The presented code undertakes a series of tasks to process and prepare image datasets for machine learning:

Initially, it loads images categorized as "NORMAL" and "PNEUMONIA," returning normalized and resized versions. This functionality operates by accepting a directory path pointing to the image folder and generating two lists: one containing the images and another containing their corresponding class labels.

Subsequently, the code combines the training and testing datasets into two arrays, segregating images and labels.

To provide a visual representation of the dataset's class distribution, a bar chart is generated. Following this, the data undergoes a split into training and testing sets utilizing the train_test_split() function from scikit-learn.

The code introduces the plot_gallery() function, designed to exhibit a gallery of images from the training set along with their associated class labels.

The script defines two image datasets utilizing TensorFlow's image_dataset_from_directory function. One dataset is allocated for training purposes, while the other serves as the testing dataset. Images in both datasets are resized to dimensions of 64x64 pixels. A preview of images from the training dataset, accompanied by their respective class labels, is then displayed.

Additionally, the pixel values of images in both datasets are normalized, ensuring they fall within the 0 to 1 range through the utilization of the normalize function.

Lastly, the process_ds_to_numpy function is introduced. This function transforms datasets into numpy arrays for integration into a machine learning model. This is achieved by iteratively traversing each image in the dataset, appending its pixel values to a list, and subsequently concatenating these lists to form numpy arrays. The resulting arrays for the training and testing datasets are denoted as x_train, y_train, x_test, and y_test.

In [ ]:
img_width, img_height = 64, 64
img_color_mode = "grayscale"  # can also be 'rgb'
classes = {0: "NORMAL", 1: "PNEUMONIA"}
n_classes = 2

train_ds = tf.keras.utils.image_dataset_from_directory(
    "./train/",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode=img_color_mode,
    batch_size=32,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "./test/",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode=img_color_mode,
    batch_size=32,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(25):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"), cmap="gray")
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.suptitle("Image Classes")
plt.show()

The provided code comprises functions for normalizing pixel values in images and converting TensorFlow datasets to numpy arrays.

The normalize function accepts an image and its label, normalizes the pixel values of the image to a range between 0 and 1, and returns the normalized image along with its label. Subsequently, the train_ds and test_ds datasets undergo normalization by mapping them to the normalize function.

Concluding the script, the process_ds_to_numpy function is employed to convert the train_ds and test_ds datasets into numpy arrays. The final line of the code determines the number of dimensions in the resulting x_train numpy array.

In [ ]:
def normalize(image, label):
    """Normalize the pixel values of the image to be between 0 and 1."""
    return tf.cast(image, tf.float32) / 255.0, label


train_ds = train_ds.map(normalize)
test_ds = test_ds.map(normalize)

In [ ]:
def process_ds_to_numpy(ds) -> tuple:
    """Returns the x, y numpy arrays from the dataset."""
    x, y = [], []
    for image, label in ds.as_numpy_iterator():
        x.append(np.array(image, dtype=np.float32))
        y.append(np.array(label, dtype=np.int32))

    return np.concatenate(x, axis=0), np.concatenate(y, axis=0)


x_train, y_train = process_ds_to_numpy(train_ds)
x_test, y_test = process_ds_to_numpy(test_ds)

n_dimensions = x_train.shape[-1]

In [ ]:
plt.imshow(x_train[0], cmap='gray')
plt.axis('off')
plt.show()

# MODELING

The ImageDataGenerator function is employed to dynamically apply diverse image transformations during model training, encompassing rotations, zooms, shifts, and flips. This generator is then configured to the training and test data through the fit method.

Introducing additional image transformations to the existing data serves to augment the size of our training dataset. Leveraging rotation, zoom, shift, and flip techniques exposes our models to varied image angles, enhancing their ability to make predictions based on both the original and augmented data. In the context of our citrus disease dataset, where diseases exhibit defining traits but manifest in numerous variations with distinct physical symptoms, generating images from existing ones facilitates the creation of diverse disease variations. This approach not only enriches the training data but also mitigates the risk of overfitting, ensuring our models are better equipped to generalize to various manifestations of the diseases.

In [ ]:
datagen = preprocessing.image.ImageDataGenerator(
       rotation_range = 20, 
       width_shift_range=0.1,  
       height_shift_range=0.1,  
       horizontal_flip = True,  
       vertical_flip = False,
)
# Fits the data to the generator.
datagen.fit(x_train)

In [ ]:
# One-hot encodes the inputs
# One-hot encoding is necessary for the model to output probabilities for each class during training and evaluation.
y_train_ohe = utils.to_categorical(y_train, n_classes)
y_test_ohe = utils.to_categorical(y_test, n_classes)

In [ ]:
""" Reshapes the arrays to have a sample size of -1, and an image size of 64x64 with one channel, and 
then expands the # dimensions of the arrays to include the channel dimension, resulting in a final shape 
of (samples, image_rows, image_cols, image_channels)""" 

for tmp in datagen.flow(x_train, y_train_ohe, batch_size=1):
    plt.imshow(cv2.resize(tmp[0].squeeze(), (64, 64)), cmap='gray')
    plt.title("Generated X-Ray")
    plt.axis("off")
    break

### Helper Functions

In the following code segment, we establish a flow iterator by utilizing the previously defined image data generator object to produce augmented images in batches of size one from the training set. Subsequently, we employ matplotlib to visualize one of the augmented images.

To enhance visualization and gain insights into the augmented data, we specifically generate plots for 25 sample images from the training set. This allows us to observe the variations introduced through augmentation, providing a comprehensive view of the diverse transformations applied to the original images.

In [ ]:
def plot_confusion_matrix(
    cm,
    target_names,
    title="Confusion matrix",
    cmap=None,
    normalize=True,
    class_results: dict = {},
):
    """
    Given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap("Blues")

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(
                j,
                i,
                "{:0.4f}".format(cm[i, j]),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black",
            )
        else:
            plt.text(
                j,
                i,
                "{:,}".format(cm[i, j]),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black",
            )

    plt.tight_layout()
    plt.ylabel("True label")
    if class_results:
        x_lab = "Predicted label\n\n"
        x_lab += f"accuracy={accuracy:0.4f}\nmisclass={misclass:0.4f}\n"
        for key, value in class_results.items():
            x_lab += f"{key}={value:0.4f}\n"
    else:
        x_lab = "Predicted label\n\n"
        x_lab += f"accuracy={accuracy:0.4f}\nmisclass={misclass:0.4f}\n"
        
    plt.xlabel(x_lab)
    plt.show()

In the code below, a helper function named compare_mlp_cnn is introduced. This function takes the following inputs:

- cnn and mlp: the trained convolutional neural network and multilayer perceptron models, respectively.
- X_test and y_test: the testing data and corresponding labels.
- labels: an optional list of class labels, defaulting to 'auto'.

The primary purpose of this function is to assess and compare the performance of the CNN and MLP models on the test data. It achieves this by making predictions for class labels and subsequently computing accuracy scores and confusion matrices. The results are then visualized using a heatmap, leveraging the seaborn library. This visualization aids in providing an intuitive representation of the comparative performance between the two models on the test data.

In [ ]:
def compare_mlp_cnn(model_1, model_2, X_test, y_test, title_1: str, title_2: str, labels='auto'):
    plt.figure(figsize=(15, 5))
    if model_1 is not None:
        yhat_model_1 = np.argmax(model_1.predict(X_test), axis=1)
        acc_model_1 = metrics.accuracy_score(y_test, yhat_model_1)
        plt.subplot(1, 2, 1)
        cm = metrics.confusion_matrix(y_test, yhat_model_1)
        cm = cm/np.sum(cm,axis=1)[:,np.newaxis]
        sns.heatmap(cm, annot=True, fmt='.2%', xticklabels=labels, yticklabels=labels)
        plt.title(f"{title_1}: {acc_model_1:.3f}")
    
    if model_2 is not None:
        yhat_model_2 = np.argmax(model_2.predict(X_test), axis=1)
        acc_model_2 = metrics.accuracy_score(y_test, yhat_model_2)
        plt.subplot(1, 2, 2)
        cm = metrics.confusion_matrix(y_test,yhat_model_2)
        cm = cm/np.sum(cm,axis=1)[:,np.newaxis]
        sns.heatmap(cm,annot=True, fmt='.2%', xticklabels=labels, yticklabels=labels)
        plt.title(f"{title_2}: {acc_model_2:.3f}")

In [12]:
# Evaluates the performance of a model during training and identifying overfitting or underfitting.
def plot_history(history):
    # plot history of the model:
    fig = plt.figure(figsize=(10, 8))
    plt.plot(history.history["loss"], label="train")
    plt.plot(history.history["val_loss"], label="test")
    plt.legend()
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss over epochs")

In [13]:
"""The summarize_net helper function is designed to provide a comprehensive summary of a neural network 
model's performance. This function takes the following inputs:

- net: the neural network model.
- x_test: the test set features.
- y_test: the test set labels.
- An optional string parameter for the title of the plot.

The function utilizes the predict method of the model to generate predictions for the test set. It then 
computes the accuracy score by comparing these predicted labels with the actual labels. Additionally, 
the function calculates the confusion matrix, normalizes it, and presents it as aheatmap using the Seaborn 
library. The title of the plot is constructed to include the accuracy score formatted to four decimal places, 
along with any optional title text provided to the function. This visual representation aids in effectively 
summarizing the model's performance on the given test set."""

def summarize_net(net, x_test, y_test, title_text=""):
    label = ["NORMAL","PNEUMONIA"]
    plt.figure(figsize=(15, 5))
    yhat = np.argmax(net.predict(x_test), axis=1)
    acc = metrics.accuracy_score(y_test, yhat)
    cm = metrics.confusion_matrix(y_test, yhat)
    cm = cm / np.sum(cm, axis=1)[:, np.newaxis]
    sns.heatmap(cm, annot=True, fmt=".2%", xticklabels=label, yticklabels=label)
    plt.title(title_text + "{:.4f}".format(acc))

# MLP

The provided code defines a 3-layer perceptron (MLP) model using Keras with the following specifications:

- The input images are flattened before being fed into the MLP.
- The first hidden layer has 30 units with a Rectified Linear Unit (ReLU) activation function.
- The second hidden layer has 15 units, also with a ReLU activation function.
- The output layer has as many units as the number of classes, employing a softmax activation function.
- The model is compiled with the RMSprop optimizer and mean squared error loss function.
- Evaluation metrics include Recall and Area Under the Curve (AUC).
- Training is performed on the training data (x_train and y_train_ohe) with a batch size of 50 and 250 epochs.
- The training history is saved in the variable model_history.
- Finally, the model summary is printed.

This model architecture and configuration are suitable for a classification task, and the choice of metrics indicates a focus on assessing both sensitivity (Recall) and overall discriminative performance (AUC). The code provides a comprehensive overview of the model's architecture, training parameters, and evaluation metrics.

In [ ]:
# Makes a 3-layer Keras MLP
mlp = models.Sequential()

# Makes the images flat for the MLP input
mlp.add(layers.Flatten())
mlp.add(layers.Dense(input_dim=1, units=30, activation="relu"))
mlp.add(layers.Dense(units=15, activation="relu"))
mlp.add(layers.Dense(n_classes))
mlp.add(layers.Activation("softmax"))

# Compiles the model
mlp.compile(
    optimizer="Adamax",
    loss="mean_squared_error",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

# Fits the model to the training data
mlp_history = mlp.fit(
    x_train,
    y_train_ohe,
    batch_size=50,
    epochs=250,
    verbose=1,
    validation_data=(x_test, y_test_ohe),
)

clear_screen()

# Prints model summary
mlp.summary()

In [ ]:
# Plots the visualization of the MLP model in a PNG image file
utils.plot_model(
    mlp,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="LR",
    expand_nested=False,
    dpi=96,
)

In [ ]:
# Evaluates the performance of an MLP model on a test set by predicting the class 
# probabilities and rounding them to obtain class predictions.

# Defines the y-prediction probability and y-prediction arrays
y_predict_proba = mlp.predict(x_test)
y_predict = np.round(y_predict_proba)

# Prints classification report
print(classification_report(y_test_ohe, y_predict))

In [ ]:
# Variables for determining the loss over epochs
epochs = mlp_history.epoch
loss = mlp_history.history["loss"]

# Plots the loss graph
plt.plot(epochs, loss)
plt.ylabel("Cost")
plt.xlabel("Epochs")
plt.title("Loss")

plt.tight_layout()

plt.show()

##### Observing the performance of the MLP with the dataset, the curve appears to...

In [ ]:
"""The provided code evaluates and visualizes the Receiver Operating Characteristic (ROC) curve and 
Area Under the Curve (AUC) for the MLP model. The ROC curve is a graphical representation illustrating 
the performance of a binary classifier system by displaying the trade-off between the true positive rate (TPR) 
and the false positive rate (FPR) as the decision threshold for classification varies.

In this run, the calculated AUC is 0.95, indicating that the model possesses a strong ability to distinguish 
between positive and negative cases. The ROC curve visually supports this interpretation by depicting a steep 
rise in the true positive rate while maintaining a low false positive rate. This pattern suggests that the model 
effectively identifies positive cases while minimizing false positives. Overall, the high AUC and the shape of the 
ROC curve imply that the MLP model performs well on the test set, demonstrating its capability to accurately classify 
the data."""

# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Plots the training versus testing graph for the MLP model
# Model history values
hist_values = list(mlp_history.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.tight_layout()

plt.show()

##### The graph has three subplots: the first subplot shows the training and validation loss versus epochs, the second subplot shows the training and validation recall versus epochs, and the third subplot shows the training and validation AUC versus epochs. 

The training versus testing graph is used to visualize the performance of the model during training. In this case, we can see that the training loss and validation loss INCREASE/DECREASE as the number of epochs INCREASE/DECREASE, indicating that the model is ***learning and improving***. The training recall and validation recall also INCREASE/DECREASE as the number of epochs INCREASE/DECREASE, indicating that the model is becoming BETER/WORSE at identifying positive cases. Finally, the training AUC and validation AUC INCREASE/DECREASE as the number of epochs INCREASE/DECREASE, indicating that the model is becoming BETER/WORSE at distinguishing between positive and negative cases.

Overall, the graph shows that the MLP model is BETTER/WORSE over time, and that it is performing GOOD/BAD on both the training and validation sets.

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict, output_dict=True
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
)

The provided code generates a confusion matrix and visualizes it using the plot_confusion_matrix function. The confusion matrix provides a detailed breakdown of the number of true positives, true negatives, false positives, and false negatives for each class. In this specific context, the matrix illustrates the correct and incorrect classifications of images for Citrus Canker and Black Spot diseases.

From the confusion matrix, it is evident that the MLP model correctly predicted 

However, there were ### instances where NORMAL/PNEUMINOA was incorrectly predicted when the true class was 

***While the overall performance of the MLP model is good in classifying the two classes, there is a slight opportunity for improvement, particularly in reducing the number of false positives and false negatives.***

Analyzing the confusion matrix in this manner provides valuable insights into the model's strengths and areas for enhancement, aiding in the refinement of the classification algorithm.

Accuracy is a measure of the overall correctness of a model's predictions, calculated as the ratio of the number of correct predictions to the total number of predictions. For this run, our initial accuracy score is 

Misclass, or misclassification, refers to the instances where a model's prediction does not match the actual class label of a data point. Misclassification can be measured using various metrics such as accuracy, precision, recall, and F1-score - but in this case we can easily determine misclass by calculating 1.0 minus the accuracy score. In our case, we observe the misclass as 

Precision is a measure of how well a model correctly identifies positive instances among the predicted positive instances. Precision is calculated as the ratio of the number of true positive predictions to the total number of positive predictions (which would be the true positive predictions plus the false positive predictions). In our case, the precision score is 

Recall, also known as sensitivity or the true positive rate, is a measure of how well a model identifies all the positive instances among the actual positive instances. Recall is calculated as the ratio of the number of true positive predictions to the total number of actual positive instances (i.e., sum of true positive and false negative predictions). Our recall score for this run was 

F1-score is the "harmonic mean" of precision and recall, providing a single value that balances the trade-off between precision and recall. It is calculated as 2 * (precision * recall) / (precision + recall). Our F1-score for this run was 

# CNN

### CNN 1

The provided code constructs a Convolutional Neural Network (CNN) named cnn1 with the following architecture:

[1] Convolutional Layers:
- Two convolutional layers with ReLU activation.
- Max-pooling layers follow each convolutional layer.

[2] Flattening:
- The output from the convolutional layers is flattened.

[3] Dense Layers:
- A dense layer with 100 units and ReLU activation on the flattened output.
- Another dense layer with 100 units and ReLU activation.
- A final dense layer with softmax activation, producing predicted probabilities for each class.

[4] Compilation:
- The model is compiled with mean squared error as the loss function.
- RMSprop is used as the optimizer.

[5] Training:
- The model is trained on the training data with a batch size of 50.
- Training occurs for 150 epochs, with validation data provided for monitoring performance during training.

This architecture is common for image classification tasks using CNNs. The use of convolutional layers helps the model learn hierarchical features from the input images, and the subsequent dense layers facilitate the final classification. The choice of activation functions, ReLU in this case, is standard for introducing non-linearity. The softmax activation in the final layer is appropriate for multi-class classification, producing probability distributions across different classes.

The training is performed over 150 epochs, and validation data is provided to assess the model's generalization performance during training. The choice of mean squared error as the loss function might be more common in regression tasks, and for classification tasks, categorical crossentropy is often used. Additionally, the choice of RMSprop as the optimizer is suitable for many CNN applications. Depending on the specific problem and dataset, further adjustments to these parameters might be considered for optimization.

In [ ]:
# Data generator to feed the CNN model cnn1 with augmented data during the training process
# Creates a CNN with convolution layer and max pooling
cnn1 = models.Sequential()
cnn1.add(
    layers.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        padding="same",
        input_shape=(img_width, img_height, n_dimensions),
    )
)

cnn1.add(Activation("relu"))
cnn1.add(
    MaxPooling2D(
        pool_size=(2, 2),
    )
)
cnn1.add(Activation("relu"))
cnn1.add(
    MaxPooling2D(
        pool_size=(2, 2),
    )
)

# Adds 1 layer on flattened output
cnn1.add(Flatten())
cnn1.add(Dense(100, activation="relu"))
cnn1.add(Dense(50, activation="relu"))
cnn1.add(Dense(n_classes, activation="softmax"))

# Compiles the model
cnn1.compile(
    optimizer="rmsprop",
    loss="mean_squared_error",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

# Fits the model to training data
history1 = cnn1.fit(
    x_train,
    y_train_ohe,
    batch_size=50,
    epochs=150,
    validation_data=(x_test, y_test_ohe),
    shuffle=True,
    verbose=1,
)

clear_screen()

# Prints model summary
cnn1.summary()

In [ ]:
# Creates a CNN with convolution layer and max pooling for use with flow generator.
cnn1_flow = models.Sequential()
cnn1_flow.add(
    layers.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        padding="same",
        input_shape=(img_width, img_height, n_dimensions),
    )
)

cnn1_flow.add(Activation("relu"))
cnn1_flow.add(
    MaxPooling2D(
        pool_size=(2, 2),
    )
)
cnn1_flow.add(Activation("relu"))
cnn1_flow.add(
    MaxPooling2D(
        pool_size=(2, 2),
    )
)

# Adds 1 layer on flattened output
cnn1_flow.add(Flatten())
cnn1_flow.add(Dense(100, activation="relu"))
cnn1_flow.add(Dense(50, activation="relu"))
cnn1_flow.add(Dense(n_classes, activation="softmax"))

# Compiles the model
cnn1_flow.compile(
    optimizer="rmsprop",
    loss="mean_squared_error",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

# Fits the model to training data using flow generator
history1_1 = cnn1_flow.fit(
    datagen.flow(x_train, y_train_ohe, batch_size=128),
    steps_per_epoch=len(x_train) // 128,
    epochs=150,
    shuffle=True,
    verbose=1,
    validation_data=(x_test, y_test_ohe),
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss", patience=10, start_from_epoch=100
        )
    ],
)

clear_screen()

# Prints model summary
cnn1_flow.summary()

In [ ]:
# Plots the graph
utils.plot_model(
    cnn1,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="LR",
    expand_nested=False,
    dpi=96,
)

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba = cnn1.predict(x_test)
y_predict = np.round(y_predict_proba)

# Prints classification report
print(classification_report(y_test_ohe, y_predict))

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba_flow = cnn1_flow.predict(x_test)
y_predict_flow = np.round(y_predict_proba_flow)

# Prints classification report
print(classification_report(y_test_ohe, y_predict_flow))

In [ ]:
# Variables for determining the loss over epochs
epochs = history1.epoch
loss = history1.history["loss"]

epochs_flow = history1_1.epoch
loss_flow = history1_1.history["loss"]

# Plots the loss graph
plt.plot(epochs, loss, label="CNN")
plt.plot(epochs_flow, loss_flow, label="CNN (Flow Generator) w/ Early Stopping")
plt.ylabel("Cost")
plt.xlabel("Epochs")
plt.title("Loss")

plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict, output_dict=True
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="CNN Confusion Matrix"
)

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict_flow, output_dict=True
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict_flow, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="CNN (With Flow Generator) Confusion Matrix"
)

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("CNN 1 - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba_flow, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("CNN 1 (Flow Generator) - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Model history values
hist_values = list(history1.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("CNN 1")
plt.tight_layout()

plt.show()

In [ ]:
# Model history values
hist_values = list(history1_1.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("CNN 1 With Flow Generator")
plt.tight_layout()

plt.show()

# CNN 1 vs MLP

In [ ]:
compare_mlp_cnn(
    cnn1,
    mlp,
    x_test,
    y_test,
    title_1="CNN 1",
    title_2="MLP",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    cnn1_flow,
    mlp,
    x_test,
    y_test,
    title_1="CNN 1 (Flow Generator)",
    title_2="MLP",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    cnn1,
    cnn1_flow,
    x_test,
    y_test,
    title_1="CNN 1",
    title_2="CNN 1 (Flow Generator)",
    labels=[classes[0], classes[1]],
)

### CNN 2

In [ ]:
# Creates a CNN with convolution layer and max pooling
cnn2 = models.Sequential()
cnn2.add(
    layers.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation="relu",
        data_format="channels_last",
        input_shape=(img_width, img_height, n_dimensions),
    )
)
cnn2.add(
    layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation="relu",
        data_format="channels_last",
        input_shape=(img_width, img_height, n_dimensions),
    )
)
cnn2.add(Activation("relu"))
cnn2.add(
    MaxPooling2D(
        pool_size=(2, 2),
        data_format="channels_last"
    )
)
cnn2.add(Activation("relu"))
cnn2.add(
    MaxPooling2D(
        pool_size=(2, 2),
        data_format="channels_last"
    )
)

# Adds dropout layer
cnn2.add(Dropout(0.2))

# Adds 1 layer on flattened output
cnn2.add(Flatten())

cnn2.add(Dense(512, activation="relu"))
cnn2.add(Dense(256, activation="relu"))
cnn2.add(Dense(128, activation="relu"))

# Adds dropout layer
cnn2.add(Dropout(0.4))

cnn2.add(Dense(n_classes, activation="sigmoid"))

# Compiles the model
cnn2.compile(
    optimizer="Adamax",
    loss="mean_squared_error",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

# Fits the model to training data
history2 = cnn2.fit(
    x_train,
    y_train_ohe,
    batch_size=50,
    epochs=150,
    validation_data=(x_test, y_test_ohe),
    shuffle=True,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss", patience=3, start_from_epoch=50
        )
    ],
)

clear_screen()

# Prints model summary
cnn2.summary()

In [ ]:
# Creates a CNN with convolution layer and max pooling for use with flow generator.
cnn2_flow = models.Sequential()
cnn2_flow.add(
    layers.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        activation="relu",
        data_format="channels_last",
        input_shape=(img_width, img_height, n_dimensions),
    )
)
cnn2_flow.add(
    layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        activation="relu",
        data_format="channels_last",
        input_shape=(img_width, img_height, n_dimensions),
    )
)
cnn2_flow.add(Activation("relu"))
cnn2_flow.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
cnn2_flow.add(Activation("relu"))
cnn2_flow.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

# Adds dropout layer
cnn2_flow.add(Dropout(0.2))

# Adds 1 layer on flattened output
cnn2_flow.add(Flatten())

# cnn2_flow.add(Dense(1024, activation="relu"))
cnn2_flow.add(Dense(512, activation="relu"))
cnn2_flow.add(Dense(256, activation="relu"))
cnn2_flow.add(Dense(128, activation="relu"))

# Adds dropout layer
cnn2_flow.add(layers.Dropout(0.4))

cnn2_flow.add(Dense(n_classes, activation="sigmoid"))

# Compiles the model
cnn2_flow.compile(
    optimizer="Adamax",
    loss="mean_squared_error",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

# Fits the model to training data using flow generator
history2_2 = cnn2_flow.fit(
    datagen.flow(x_train, y_train_ohe, batch_size=50),
    steps_per_epoch=len(x_train) // 128,
    epochs=150,
    shuffle=True,
    verbose=1,
    validation_data=(x_test, y_test_ohe),
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss",
            patience=3,
            start_from_epoch=50,
            restore_best_weights=True,
        )
    ],
)

clear_screen()

# Prints model summary
cnn2_flow.summary()

In [ ]:
# Plots the graph
utils.plot_model(
    cnn2,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="LR",
    expand_nested=False,
    dpi=96,
)

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba = cnn2.predict(x_test)
y_predict = np.round(y_predict_proba)

# Prints classification report
print(classification_report(y_test_ohe, y_predict))

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba_flow = cnn2_flow.predict(x_test)
y_predict_flow = np.round(y_predict_proba_flow)

# Prints classification report
print(classification_report(y_test_ohe, y_predict_flow, zero_division=0))

In [ ]:
# Variables for determining the loss over epochs
epochs = history2.epoch
loss = history2.history["loss"]

epochs_flow = history2_2.epoch
loss_flow = history2_2.history["loss"]

# Plots the loss graph
plt.plot(epochs, loss, label="CNN w/ Early Stopping")
plt.plot(epochs_flow, loss_flow, label="CNN (Flow Generator) w/ Early Stopping")
plt.ylabel("Cost")
plt.xlabel("Epochs")
plt.title("Loss")

plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict, output_dict=True
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="CNN Confusion Matrix"
)

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict_flow, output_dict=True, zero_division=0
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict_flow, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="CNN (With Flow Generator) Confusion Matrix"
)

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("CNN 2 - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba_flow, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("CNN 2 (Flow Generator) - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Model history values
hist_values = list(history2.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("CNN 2")
plt.tight_layout()

plt.show()

In [ ]:
# Model history values
hist_values = list(history2_2.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("CNN 2 With Flow Generator")
plt.tight_layout()

plt.show()

# CNN 2 vs MLP

In [ ]:
compare_mlp_cnn(
    cnn2,
    mlp,
    x_test,
    y_test,
    title_1="CNN 2",
    title_2="MLP",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    cnn2_flow,
    mlp,
    x_test,
    y_test,
    title_1="CNN 2 (Flow Generator)",
    title_2="MLP",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    cnn2,
    cnn2_flow,
    x_test,
    y_test,
    title_1="CNN 2",
    title_2="CNN 2 (Flow Generator)",
    labels=[classes[0], classes[1]],
)

### CNN 3

In [ ]:
# Use Kaiming He to regularize ReLU layers: https://arxiv.org/pdf/1502.01852.pdf
# Use Glorot/Bengio for linear/sigmoid/softmax: http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf
l2_lambda = 1e-4
cnn3 = Sequential()

cnn3.add(
    Conv2D(
        filters=32,
        input_shape=(img_width, img_height, n_dimensions),
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)

cnn3.add(
    Conv2D(
        filters=32,
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)
cnn3.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

cnn3.add(
    Conv2D(
        filters=64,
        input_shape=(img_width, img_width, n_dimensions),
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)

cnn3.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
    )
)
cnn3.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

cnn3.add(
    Conv2D(
        filters=128,
        input_shape=(img_width, img_width, n_dimensions),
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)  # more compact syntax

cnn3.add(
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)


# add one layer on flattened output
cnn3.add(Flatten())
cnn3.add(Dropout(0.25))  # add some dropout for regularization after conv layers
cnn3.add(
    Dense(
        128,
        activation="relu",
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
    )
)
cnn3.add(Dropout(0.5))  # add some dropout for regularization, again!
cnn3.add(
    Dense(
        n_classes,
        activation="sigmoid",
        kernel_initializer="glorot_uniform",
        kernel_regularizer=l2(l2_lambda),
    )
)

# Let's train the model
cnn3.compile(
    optimizer="Adamax",
    loss="binary_crossentropy",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

history3 = cnn3.fit(
    x_train,
    y_train_ohe,
    steps_per_epoch=len(x_train) // 128,
    epochs=150,
    verbose=1,
    validation_data=(x_test, y_test_ohe),
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss",
            patience=3,
            start_from_epoch=50,
            restore_best_weights=True,
        )
    ],
)

clear_screen()

# Prints model summary
cnn3.summary()

In [ ]:
# Use Kaiming He to regularize ReLU layers: https://arxiv.org/pdf/1502.01852.pdf
# Use Glorot/Bengio for linear/sigmoid/softmax: http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf
l2_lambda = 1e-4
cnn3_flow = Sequential()

cnn3_flow.add(
    Conv2D(
        filters=32,
        input_shape=(img_width, img_height, n_dimensions),
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)

cnn3_flow.add(
    Conv2D(
        filters=32,
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)
cnn3_flow.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

cnn3_flow.add(
    Conv2D(
        filters=64,
        input_shape=(img_width, img_width, n_dimensions),
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)

cnn3_flow.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
    )
)
cnn3_flow.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

cnn3_flow.add(
    Conv2D(
        filters=128,
        input_shape=(img_width, img_width, n_dimensions),
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)

cnn3_flow.add(
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
        padding="same",
        activation="relu",
        data_format="channels_last",
    )
)


# add one layer on flattened output
cnn3_flow.add(Flatten())
cnn3_flow.add(Dropout(0.25))  # add some dropout for regularization after conv layers
cnn3_flow.add(
    Dense(
        128,
        activation="relu",
        kernel_initializer="he_uniform",
        kernel_regularizer=l2(l2_lambda),
    )
)
cnn3_flow.add(Dropout(0.5))  # add some dropout for regularization, again!
cnn3_flow.add(
    Dense(
        n_classes,
        activation="softmax",
        kernel_initializer="glorot_uniform",
        kernel_regularizer=l2(l2_lambda),
    )
)

# Let's train the model
cnn3_flow.compile(
    optimizer="Adamax",
    loss="binary_crossentropy",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

history3_3 = cnn3_flow.fit(
    datagen.flow(x_train, y_train_ohe, batch_size=128),
    steps_per_epoch=len(x_train) // 128,
    epochs=150,
    verbose=1,
    shuffle=True,
    validation_data=(x_test, y_test_ohe),
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss",
            patience=3,
            start_from_epoch=50,
            restore_best_weights=True,
        )
    ],
)

clear_screen()

# Prints model summary
cnn3.summary()

In [ ]:
# Plots the graph
utils.plot_model(
    cnn3,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="LR",
    expand_nested=False,
    dpi=96,
)

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba = cnn3.predict(x_test)
y_predict = np.round(y_predict_proba)

# Prints classification report
print(classification_report(y_test_ohe, y_predict))

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba_flow = cnn3_flow.predict(x_test)
y_predict_flow = np.round(y_predict_proba_flow)

# Prints classification report
print(classification_report(y_test_ohe, y_predict_flow, zero_division=0))

In [ ]:
# Variables for determining the loss over epochs
epochs = history3.epoch
loss = history3.history["loss"]

epochs_flow = history3_3.epoch
loss_flow = history3_3.history["loss"]

# Plots the loss graph
plt.plot(epochs, loss, label="CNN w/ Early Stopping")
plt.plot(epochs_flow, loss_flow, label="CNN (Flow Generator) w/ Early Stopping")
plt.ylabel("Cost")
plt.xlabel("Epochs")
plt.title("Loss")

plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict, output_dict=True
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="CNN Confusion Matrix"
)

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict_flow, output_dict=True, zero_division=0
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict_flow, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="CNN (With Flow Generator) Confusion Matrix"
)

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("CNN 3 - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba_flow, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("CNN 3 (Flow Generator) - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Model history values
hist_values = list(history3.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("CNN 3 With Flow Generator")
plt.tight_layout()

plt.show()


In [ ]:
# Model history values
hist_values = list(history3_3.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("CNN 3 With Flow Generator")
plt.tight_layout()

plt.show()

# CNN 3 vs MLP

In [ ]:
compare_mlp_cnn(
    cnn3,
    mlp,
    x_test,
    y_test,
    title_1="CNN 3",
    title_2="MLP",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    cnn3_flow,
    mlp,
    x_test,
    y_test,
    title_1="CNN 3 (Flow Generator)",
    title_2="MLP",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    cnn3,
    cnn3_flow,
    x_test,
    y_test,
    title_1="CNN 3",
    title_2="CNN 3 (Flow Generator)",
    labels=[classes[0], classes[1]],
)

### CNN 4: Using a LeNet Architecture

In [ ]:
input_holder = Input(shape=(img_width, img_height, n_dimensions))

# start with a conv layer
x = Conv2D(
    filters=32,
    input_shape=(img_width, img_height, n_dimensions),
    kernel_size=(3, 3),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(input_holder)

x = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)

x = Conv2D(
    filters=32,
    kernel_size=(3, 3),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(x)

x_split = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)

x = Conv2D(
    filters=64,
    kernel_size=(1, 1),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(x_split)

x = Conv2D(
    filters=64,
    kernel_size=(3, 3),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(x)

x = Conv2D(
    filters=32,
    kernel_size=(1, 1),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(x)

# now add back in the split layer, x_split (residual added in)
x = Add()([x, x_split])
x = Activation("relu")(x)

x = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)

x = Flatten()(x)
x = Dropout(0.25)(x)
x = Dense(256)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(n_classes)(x)
x = Activation("softmax")(x)

cnn4 = Model(inputs=input_holder, outputs=x)

cnn4.compile(
    optimizer="Adamax",
    loss="binary_crossentropy",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

history4 = cnn4.fit(
    x_train,
    y_train_ohe,
    batch_size=50,
    epochs=150,
    verbose=1,
    validation_data=(x_test, y_test_ohe),
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss",
            patience=3,
            start_from_epoch=50,
            restore_best_weights=True,
        )
    ],
)

clear_screen()

# Prints model summary
cnn4.summary()

In [ ]:
input_holder = Input(shape=(img_width, img_height, n_dimensions))

# start with a conv layer
x = Conv2D(
    filters=32,
    input_shape=(img_width, img_height, n_dimensions),
    kernel_size=(3, 3),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(input_holder)

x = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)

x = Conv2D(
    filters=32,
    kernel_size=(3, 3),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(x)

x_split = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)

x = Conv2D(
    filters=64,
    kernel_size=(1, 1),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(x_split)

x = Conv2D(
    filters=64,
    kernel_size=(3, 3),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(x)

x = Conv2D(
    filters=32,
    kernel_size=(1, 1),
    kernel_initializer="he_uniform",
    kernel_regularizer=l2(l2_lambda),
    padding="same",
    activation="relu",
    data_format="channels_last",
)(x)

# now add back in the split layer, x_split (residual added in)
x = Add()([x, x_split])
x = Activation("relu")(x)

x = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(x)

x = Flatten()(x)
x = Dropout(0.25)(x)
x = Dense(256)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(n_classes)(x)
x = Activation("softmax")(x)

cnn4_flow = Model(inputs=input_holder, outputs=x)

cnn4_flow.compile(
    optimizer="Adamax",
    loss="binary_crossentropy",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

history4_4 = cnn4_flow.fit(
    datagen.flow(x_train, y_train_ohe, batch_size=128),
    steps_per_epoch=len(x_train) // 128,
    batch_size=50,
    epochs=150,
    verbose=1,
    validation_data=(x_test, y_test_ohe),
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss",
            patience=3,
            start_from_epoch=50,
            restore_best_weights=True,
        )
    ],
)

clear_screen()

# Prints model summary
cnn4_flow.summary()

In [ ]:
# Plots the graph
utils.plot_model(
    cnn4,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="LR",
    expand_nested=False,
    dpi=96,
)

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba = cnn4.predict(x_test)
y_predict = np.round(y_predict_proba)

# Prints classification report
print(classification_report(y_test_ohe, y_predict))

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba_flow = cnn4_flow.predict(x_test)
y_predict_flow = np.round(y_predict_proba_flow)

# Prints classification report
print(classification_report(y_test_ohe, y_predict_flow, zero_division=0))

In [ ]:
# Variables for determining the loss over epochs
epochs = history4.epoch
loss = history4.history["loss"]

epochs_flow = history4_4.epoch
loss_flow = history4_4.history["loss"]

# Plots the loss graph
plt.plot(epochs, loss, label="CNN w/ Early Stopping")
plt.plot(epochs_flow, loss_flow, label="CNN (Flow Generator) w/ Early Stopping")
plt.ylabel("Cost")
plt.xlabel("Epochs")
plt.title("Loss")

plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict, output_dict=True
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="CNN Confusion Matrix"
)

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict_flow, output_dict=True, zero_division=0
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict_flow, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="CNN (With Flow Generator) Confusion Matrix"
)

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("CNN 4 - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba_flow, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("CNN 4 (Flow Generator) - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Model history values
hist_values = list(history4.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("CNN 4")
plt.tight_layout()

plt.show()

In [ ]:
# Model history values
hist_values = list(history4_4.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("CNN 4 With Flow Generator")
plt.tight_layout()
plt.show()

# CNN 4 vs MLP

In [ ]:
compare_mlp_cnn(
    cnn4,
    mlp,
    x_test,
    y_test,
    title_1="CNN 4",
    title_2="MLP",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    cnn4_flow,
    mlp,
    x_test,
    y_test,
    title_1="CNN 4 (Flow Generator)",
    title_2="MLP",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    cnn4,
    cnn4_flow,
    x_test,
    y_test,
    title_1="CNN 4",
    title_2="CNN 4 (Flow Generator)",
    labels=[classes[0], classes[1]],
)

# GRADUATE ANALYSIS

In [ ]:
# Reset images for compatibility with Resnet
img_width, img_height = 64, 64
img_color_mode = "rgb"
classes = {0: "NORMAL", 1: "PNEUMONIA"}
n_classes = 2

train_ds = tf.keras.utils.image_dataset_from_directory(
    "./train/",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode=img_color_mode,
    batch_size=32,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "./test/",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode=img_color_mode,
    batch_size=32,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)


def normalize(image, label):
    """Normalize the pixel values of the image to be between 0 and 1."""
    return tf.cast(image, tf.float32) / 255.0, label


train_ds = train_ds.map(normalize)
test_ds = test_ds.map(normalize)


def process_ds_to_numpy(ds) -> tuple:
    """Returns the x, y numpy arrays from the dataset."""
    x, y = [], []
    for image, label in ds.as_numpy_iterator():
        x.append(np.array(image, dtype=np.float32))
        y.append(np.array(label, dtype=np.int32))

    return np.concatenate(x, axis=0), np.concatenate(y, axis=0)


x_train, y_train = process_ds_to_numpy(train_ds)
x_test, y_test = process_ds_to_numpy(test_ds)

n_dimensions = x_train.shape[-1]

datagen = preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
)
# Fits the data to the generator.
datagen.fit(x_train)

# One-hot encodes the inputs
y_train_ohe = utils.to_categorical(y_train, n_classes)
y_test_ohe = utils.to_categorical(y_test, n_classes)

In [ ]:
# Recreate our previous CNN for pre-training
base_model = models.Sequential()
base_model.add(
    layers.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        padding="same",
        input_shape=(img_width, img_height, n_dimensions),
    )
)

base_model.add(Activation("relu"))
base_model.add(
    MaxPooling2D(
        pool_size=(2, 2),
    )
)
base_model.add(Activation("relu"))
base_model.add(
    MaxPooling2D(
        pool_size=(2, 2),
    )
)

# Adds 1 layer on flattened output
base_model.add(Flatten())
base_model.add(Dense(100, activation="relu"))
base_model.add(Dense(50, activation="relu"))
base_model.add(Dense(n_classes, activation="softmax"))

# Compiles the model
base_model.compile(
    optimizer="rmsprop",
    loss="mean_squared_error",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

# Fits the model to training data
base_model_history = base_model.fit(
    x_train,
    y_train_ohe,
    batch_size=50,
    epochs=150,
    validation_data=(x_test, y_test_ohe),
    shuffle=True,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss", patience=3, start_from_epoch=20
        )
    ],
)

# Save model for pre-training
keras.models.save_model(base_model, "base_model.h5")

clear_screen()

# Prints model summary
base_model.summary()

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba = base_model.predict(x_test)
y_predict = np.round(y_predict_proba)

# Prints classification report
print(classification_report(y_test_ohe, y_predict, zero_division=0))

In [ ]:
pretrained_model = keras.models.load_model("base_model.h5")

# Set 1st layer to not trainable (e.g., "Remove the top")
pretrained_model.layers[0].trainable = False

# Implement transfer learning
inputs = keras.Input(shape=(64, 64, 3))

x = pretrained_model.layers[1].output

x = layers.Conv2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    activation="relu",
    data_format="channels_last",
    input_shape=(img_width, img_height, n_dimensions),
)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(8, activation="relu")(x)

outputs = keras.layers.Dense(2, activation="sigmoid")(x)
new_model = keras.Model(inputs=pretrained_model.input, outputs=outputs)

new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

new_model_history = new_model.fit(
    x_train,
    y_train_ohe,
    steps_per_epoch=len(x_train) // 128,
    batch_size=50,
    epochs=100,
    validation_data=(x_test, y_test_ohe),
    shuffle=True,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss", patience=3, start_from_epoch=25
        )
    ],
)

clear_screen()

# Displays model summary
new_model.summary()

In [ ]:
pretrained_model = keras.models.load_model("base_model.h5")

# Set 1st layer to not trainable (e.g., "Remove the top")
pretrained_model.layers[0].trainable = False

# Implement transfer learning
inputs = keras.Input(shape=(64, 64, 3))

x = pretrained_model.layers[1].output

x = layers.Conv2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    activation="relu",
    data_format="channels_last",
    input_shape=(img_width, img_height, n_dimensions),
)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(16, activation="relu")(x)

outputs = keras.layers.Dense(2, activation="sigmoid")(x)
new_model_flow = keras.Model(inputs=pretrained_model.input, outputs=outputs)

new_model_flow.compile(
    optimizer=keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

new_model_flow_history = new_model_flow.fit(
    datagen.flow(x_train, y_train_ohe, batch_size=128),
    steps_per_epoch=len(x_train) // 128,
    batch_size=50,
    epochs=100,
    validation_data=(x_test, y_test_ohe),
    shuffle=True,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss", patience=3, start_from_epoch=25
        )
    ],
)

clear_screen()

# Displays model summary
new_model_flow.summary()

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba = new_model.predict(x_test)
y_predict = np.round(y_predict_proba)

# Prints classification report
print(classification_report(y_test_ohe, y_predict, zero_division=0))

In [ ]:
# Defines the y-prediction probability and y-prediction arrays
y_predict_proba_flow = new_model_flow.predict(x_test)
y_predict_flow = np.round(y_predict_proba_flow)

# Prints classification report
print(classification_report(y_test_ohe, y_predict_flow, zero_division=0))

In [ ]:
# Variables for determining the loss over epochs
epochs = new_model_history.epoch
loss = new_model_history.history["loss"]

epochs_flow = new_model_flow_history.epoch
loss_flow = new_model_flow_history.history["loss"]

# Plots the loss graph
plt.plot(epochs, loss, label="Transfer Learn CNN w/ Early Stopping")
plt.plot(epochs_flow, loss_flow, label="Transfer Learn CNN (Flow Generator) w/ Early Stopping")
plt.ylabel("Cost")
plt.xlabel("Epochs")
plt.title("Loss")

plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict, output_dict=True
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="Transfer Learn CNN Confusion Matrix"
)

In [ ]:
class_report = metrics.classification_report(
    y_test_ohe, y_predict_flow, output_dict=True, zero_division=0
)

plot_confusion_matrix(
    metrics.confusion_matrix(
        np.argmax(y_test_ohe, axis=1), np.argmax(y_predict_flow, axis=1)
    ),
    [classes[0], classes[1]],
    normalize=False,
    class_results=class_report["weighted avg"],
    title="Transfer Learn CNN (With Flow Generator) Confusion Matrix"
)

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("Transfer Learn CNN - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Variables for determining the ROC/AUC
fpr, tpr, threshold = roc_curve(y_test, np.argmax(y_predict_proba_flow, axis=1))
roc_auc = auc(fpr, tpr)

# Plots the ROC and AUC graph
plt.title("Transfer Learn CNN (Flow Generator ) - Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([-.01, 1.01])
plt.ylim([-.01, 1.01])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
# Model history values
hist_values = list(new_model_history.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("Transfer Learn CNN Model")
plt.tight_layout()

plt.show()

In [ ]:
# Model history values
hist_values = list(new_model_flow_history.history.values())

# Variables for plotting the training versus testing
train_loss   = hist_values[0]
train_recall = hist_values[1]
train_auc    = hist_values[2]
val_loss     = hist_values[3]
val_recall   = hist_values[4]
val_auc      = hist_values[5]

# Plots the training versus testing graph
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, (1, 2))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(train_recall, label="Training Recall")
plt.plot(val_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(train_auc, label="Training AUC")
plt.plot(val_auc, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

plt.suptitle("Transfer Learn CNN With Flow Generator")
plt.tight_layout()

plt.show()

# CNN vs MLP: New Model vs Base Model

In [ ]:
compare_mlp_cnn(
    new_model,
    base_model,
    x_test,
    y_test,
    title_1="New Model",
    title_2="Base Model",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    new_model_flow,
    base_model,
    x_test,
    y_test,
    title_1="New Model (Flow Generator)",
    title_2="Base Model",
    labels=[classes[0], classes[1]],
)

### ResNet50

In [ ]:
# Transfer learning using Resnet50
res50_base = keras.applications.ResNet50(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(64, 64, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

res50_base.trainable = False

inputs = keras.Input(shape=(64, 64, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = res50_base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(2, activation="softmax")(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(2, activation="sigmoid")(x)
res50 = keras.Model(inputs, outputs)

res50.compile(
    optimizer=keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

res50_history = res50.fit(
    x_train,
    y_train_ohe,
    steps_per_epoch=len(x_train) // 128,
    batch_size=50,
    epochs=200,
    validation_data=(x_test, y_test_ohe),
    shuffle=True,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss", patience=3, start_from_epoch=3
        )
    ],
)

clear_screen()

# Displays model summary
res50.summary()

In [ ]:
# Use transfer learning with ResNet50
res50_base = keras.applications.ResNet50(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(64, 64, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

res50_base.trainable = False

inputs = keras.Input(shape=(64, 64, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = res50_base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(2, activation="softmax")(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(2)(x)
res50_flow = keras.Model(inputs, outputs)

res50_flow.compile(
    optimizer=keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=[keras.metrics.Recall(), keras.metrics.AUC()],
)

res50_flow_history = res50_flow.fit(
    datagen.flow(x_train, y_train_ohe, batch_size=50),
    steps_per_epoch=len(x_train) // 128,
    batch_size=50,
    epochs=200,
    validation_data=(x_test, y_test_ohe),
    shuffle=True,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(
            monitor="val_loss", patience=3, start_from_epoch=20
        )
    ],
)

clear_screen()

res50_flow.summary()

# CNN vs MLP: New Model (ResNet50) vs Base Model 

In [ ]:
compare_mlp_cnn(
    res50,
    base_model,
    x_test,
    y_test,
    title_1="New Model (ResNet 50)",
    title_2="Base Model",
    labels=[classes[0], classes[1]],
)

In [ ]:
compare_mlp_cnn(
    res50_flow,
    base_model,
    x_test,
    y_test,
    title_1="New Model (ResNet 50) Flow Generator",
    title_2="Base Model",
    labels=[classes[0], classes[1]],
)